In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("yarn").setAppName("Lab 3")
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
# display my SparkContext information
sc

<SparkContext master=yarn appName=Lab 3>

In [3]:
# input set 1:
# M = [(1, 1, 1), (1, 2, 3), (2, 1, 2), (2, 2, 4), (3, 1, 6), (3, 2, 5)]
# N = [(1, 1, 8), (1, 2, 10), (2, 1, 7), (2, 2, 9)]

# input set 2:
# M = [(1, 1, 1), (1, 2, 2), (2, 1, 3), (2, 2, 4), (3, 1, 5), (3, 2, 6)]
# N = [(1, 1, 7), (1, 2, 8), (2, 1, 9), (2, 2, 10)]

# input set 3:
M = [(1, 1, 1), (1, 2, 2), (2, 1, 3), (2, 2, 4), (3, 1, 5), (3, 2, 6)]
N = [(1, 1, 1), (1, 2, 2), (1, 3, 3), (1, 4, 4), (2, 1, 5), (2, 2, 6), (2, 3, 7), (2, 4, 8)]

In [4]:
m_dist = sc.parallelize(M)
m_dist.collect()

[(1, 1, 1), (1, 2, 2), (2, 1, 3), (2, 2, 4), (3, 1, 5), (3, 2, 6)]

In [5]:
n_dist = sc.parallelize(N)
n_dist.collect()

[(1, 1, 1),
 (1, 2, 2),
 (1, 3, 3),
 (1, 4, 4),
 (2, 1, 5),
 (2, 2, 6),
 (2, 3, 7),
 (2, 4, 8)]

In [6]:
rdd_m = m_dist.map(lambda m_ij: (m_ij[1], (m_ij[0], m_ij[2])))
rdd_m.collect()

[(1, (1, 1)), (2, (1, 2)), (1, (2, 3)), (2, (2, 4)), (1, (3, 5)), (2, (3, 6))]

In [8]:
rdd_n = n_dist.map(lambda n_jk: (n_jk[0], (n_jk[1], n_jk[2])))
rdd_n.collect()

[(1, (1, 1)),
 (1, (2, 2)),
 (1, (3, 3)),
 (1, (4, 4)),
 (2, (1, 5)),
 (2, (2, 6)),
 (2, (3, 7)),
 (2, (4, 8))]

In [9]:
rdd_join = rdd_m.join(rdd_n)
rdd_join.collect()

[(1, ((1, 1), (1, 1))),
 (1, ((1, 1), (2, 2))),
 (1, ((1, 1), (3, 3))),
 (1, ((1, 1), (4, 4))),
 (1, ((2, 3), (1, 1))),
 (1, ((2, 3), (2, 2))),
 (1, ((2, 3), (3, 3))),
 (1, ((2, 3), (4, 4))),
 (1, ((3, 5), (1, 1))),
 (1, ((3, 5), (2, 2))),
 (1, ((3, 5), (3, 3))),
 (1, ((3, 5), (4, 4))),
 (2, ((2, 4), (1, 5))),
 (2, ((2, 4), (2, 6))),
 (2, ((2, 4), (3, 7))),
 (2, ((2, 4), (4, 8))),
 (2, ((3, 6), (1, 5))),
 (2, ((3, 6), (2, 6))),
 (2, ((3, 6), (3, 7))),
 (2, ((3, 6), (4, 8))),
 (2, ((1, 2), (1, 5))),
 (2, ((1, 2), (2, 6))),
 (2, ((1, 2), (3, 7))),
 (2, ((1, 2), (4, 8)))]

In [10]:
rdd_map = rdd_join.map(lambda row: ((row[1][0][0], row[1][1][0]), row[1][0][1]*row[1][1][1]))
rdd_map.collect()

[((3, 1), 5),
 ((3, 2), 10),
 ((3, 3), 15),
 ((3, 4), 20),
 ((1, 1), 1),
 ((1, 2), 2),
 ((1, 3), 3),
 ((1, 4), 4),
 ((2, 1), 3),
 ((2, 2), 6),
 ((2, 3), 9),
 ((2, 4), 12),
 ((1, 1), 10),
 ((1, 2), 12),
 ((1, 3), 14),
 ((1, 4), 16),
 ((2, 1), 20),
 ((2, 2), 24),
 ((2, 3), 28),
 ((2, 4), 32),
 ((3, 1), 30),
 ((3, 2), 36),
 ((3, 3), 42),
 ((3, 4), 48)]

In [11]:
rdd = rdd_map.reduceByKey(lambda a, b: a+b)
result = sorted(rdd.collect())

In [12]:
result

[((1, 1), 11),
 ((1, 2), 14),
 ((1, 3), 17),
 ((1, 4), 20),
 ((2, 1), 23),
 ((2, 2), 30),
 ((2, 3), 37),
 ((2, 4), 44),
 ((3, 1), 35),
 ((3, 2), 46),
 ((3, 3), 57),
 ((3, 4), 68)]